## Facebook chat friend emulator

This is an LSTM RNN for producing sentences in the style of a given facebook friend, based on our message history

Apologies in advance to my guinea-pig, Dmitri. I censored the ramblings of your robot self to make sure you didn't say anything too outrageous!

TO DO : create chat-bot with a message-response sequence-to-sequence model

In [1]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.models import load_model
import keras.utils as ku 
import json
import glob
from random import sample

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [2]:
#Name of friend to emulate and some hyperparameters
friend = 'Dmitri'
num_messages = 7000    #randomly sample n messages from the friend's corpus
max_length = 20         #trim all messages to this many words. Most messages are short, and longer lengthen the training time significantly
lstm_size=100       #Size of the  LSTM layer

In [3]:
def generate_corpus(friend, num_messages, max_length):

    PATH_TO_CONV = glob.glob(f'data/messages/inbox/{friend}*/message.json')[0]  #data has weird names but they start with first name so match
    
    with open(PATH_TO_CONV) as f:
        data = json.load(f)
                                
    data = pd.DataFrame(data['messages'])

    def rename(name):
        if name=='Simon Roberts':
            return 'Me'
        else:
            return friend
        
    def trim_message(message):
        trimmed = str(message).split(' ')[:max_length]
        return ' '.join(trimmed)
    
    data['sender_name'] = data['sender_name'].apply(rename)   #rename senders to 'Me' and 'First Name'
    data['content'] = data[data['content'].apply(type)==str]['content'] #Only use messages which are strings (so just numbers are dropped)
    
    messages = data[data['sender_name']==friend]['content'].apply(trim_message)  #trim messages to N words

    def clean_text(txt):
        txt=str(txt)
        txt = "".join(v for v in txt if v not in string.punctuation).lower()
        txt = txt.encode("utf8").decode("ascii",'ignore')
        return txt 

    return [clean_text(message) for message in sample(list(messages), num_messages)]  #Gets N random messages

In [4]:
#Generate the corpus, and look at a few examples
corpus = generate_corpus(friend, num_messages, max_length)
sample(corpus, 5)

['would you still be as convinced then',
 'and making record highs at exponential pace while ether rallies slowly',
 'and tower of london in the background',
 'is def not cos im taking it as given ',
 'but im all for pubg']

In [5]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [6]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [7]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 16, input_length=input_len))    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(lstm_size))            #Larger vocab probably required larger LSTM layer
    model.add(Dropout(0.1))    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 23, 16)            99392     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6212)              627412    
Total params: 773,604
Trainable params: 773,604
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(predictors, label, epochs=50, verbose=1, batch_size = 128)

Epoch 1/50
42510/42510 [==============================] - 10s 244us/step - loss: 7.0727
Epoch 2/50
42510/42510 [==============================] - 9s 203us/step - loss: 6.6802
Epoch 3/50
42510/42510 [==============================] - 9s 202us/step - loss: 6.5955
Epoch 4/50
42510/42510 [==============================] - 8s 191us/step - loss: 6.5106
Epoch 5/50
42510/42510 [==============================] - 8s 192us/step - loss: 6.4133
Epoch 6/50
42510/42510 [==============================] - 8s 197us/step - loss: 6.3126
Epoch 7/50
42510/42510 [==============================] - 8s 199us/step - loss: 6.2046
Epoch 8/50
42510/42510 [==============================] - 9s 203us/step - loss: 6.0884
Epoch 9/50
42510/42510 [==============================] - 8s 191us/step - loss: 5.9738
Epoch 10/50
42510/42510 [==============================] - 8s 190us/step - loss: 5.8628
Epoch 11/50
42510/42510 [==============================] - 8s 189us/step - loss: 5.7640
Epoch 12/50
42510/42510 [===============

In [9]:
model.save(f'{friend}_{num_messages}messages_{max_length}words_{lstm_size}lstm_model.h5')

In [10]:
# A function to generate styled sentences based on a seed phrase
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text

In [23]:
#Let's see what he sounds like for some different starting words/phrases!
texts = ['will you', 'I dont', 'when can', 'I can', 'obviously']

for text in texts:
    print(f'Robo-{friend}: {generate_text(text, 20, model,max_sequence_len)}\n')

Robo-Dmitri: will you be a pilot to strike up to speed style transactions to be fresh for it and seriously first legs a

Robo-Dmitri: I dont think it was spam to be in the meantime etc etc etc etc etc etc etc etc etc etc etc

Robo-Dmitri: when can do it for a lot of the middle of creativity and its like walter tense to be honest in the

Robo-Dmitri: I can do it for a danger of the best artistic experiences of the interviewer tiki taka in dortmund and br in

Robo-Dmitri: obviously the point is just mostly pure arbitrage between dem p i circulated a dry heat ranging from 90120 f 3249



This actually sounds a lot like Dmitri!

To improve, smileys, standard texts like 'You sent a photo', etc. should either be removed or displayed in their entirety. As it is, we have their artifacts 'p' and 'D' in the results.